In [25]:
# importando libs
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
from datetime import datetime
from datetime import timedelta
import yfinance as yf
import pickle
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json
from prophet import plot
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [26]:
DAYS_PREDICT = 15

In [27]:
date_today = datetime.today().strftime("%Y-%m-%d")
data_init = "2014-01-01"

df_eth = yf.download("ETH-USD", data_init, date_today)

df_eth.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-11-09,308.644989,329.451996,307.056000,320.884003,320.884003,893249984
2017-11-10,320.670990,324.717987,294.541992,299.252991,299.252991,885985984
2017-11-11,298.585999,319.453003,298.191986,314.681000,314.681000,842300992
2017-11-12,314.690002,319.153015,298.513000,307.907990,307.907990,1613479936
2017-11-13,307.024994,328.415009,307.024994,316.716003,316.716003,1041889984


In [28]:
df_eth.reset_index(inplace=True)
df_eth.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
1768,2022-09-12,1762.084839,1778.163452,1698.293457,1713.765259,1713.765259,17688391310
1769,2022-09-13,1713.962524,1745.779053,1564.031860,1580.787964,1580.787964,23066821734
1770,2022-09-14,1574.858154,1642.157227,1564.031860,1634.755005,1634.755005,17897150206
1771,2022-09-15,1635.083008,1648.945679,1466.139404,1471.693481,1471.693481,26946275878
1772,2022-09-16,1471.928345,1480.564209,1415.581299,1432.447754,1432.447754,16764804299


In [29]:
df = df_eth[["Date", "Adj Close"]]
df.rename(columns= { "Date": "ds", "Adj Close": "y" }, inplace=True)

In [30]:
df.tail()

,ds,y
1768,2022-09-12,1713.765259
1769,2022-09-13,1580.787964
1770,2022-09-14,1634.755005
1771,2022-09-15,1471.693481
1772,2022-09-16,1432.447754


In [31]:
assert df.isnull().sum().sum() == 0
assert df['ds'].isnull().sum() == 0
assert df['y'].isnull().sum() == 0

In [32]:
df.dtypes

ds    datetime64[ns]
y            float64
dtype: object

In [33]:
model = Prophet(seasonality_mode="multiplicative")
model.fit(df)

00:34:07 - cmdstanpy - INFO - Chain [1] start processing
00:34:08 - cmdstanpy - INFO - Chain [1] done processing


In [34]:
df_future = model.make_future_dataframe(periods=DAYS_PREDICT)
df_future.tail()

,ds
1783,2022-09-27
1784,2022-09-28
1785,2022-09-29
1786,2022-09-30
1787,2022-10-01


In [35]:
# predict 
predict = model.predict(df_future)
predict

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2017-11-09,369.681878,249.718003,706.429474,369.681878,369.681878,0.293247,0.293247,0.293247,0.000628,0.000628,0.000628,0.292620,0.292620,0.292620,0.0,0.0,0.0,478.090011
1,2017-11-10,375.691854,254.778433,713.370543,375.691854,375.691854,0.287531,0.287531,0.287531,-0.005412,-0.005412,-0.005412,0.292943,0.292943,0.292943,0.0,0.0,0.0,483.714866
2,2017-11-11,381.701831,251.973710,728.538843,381.701831,381.701831,0.290996,0.290996,0.290996,-0.002031,-0.002031,-0.002031,0.293028,0.293028,0.293028,0.0,0.0,0.0,492.775666
3,2017-11-12,387.711808,263.849504,734.712819,387.711808,387.711808,0.293742,0.293742,0.293742,0.000805,0.000805,0.000805,0.292937,0.292937,0.292937,0.0,0.0,0.0,501.599033
4,2017-11-13,393.721785,271.537007,726.555905,393.721785,393.721785,0.292208,0.292208,0.292208,-0.000517,-0.000517,-0.000517,0.292726,0.292726,0.292726,0.0,0.0,0.0,508.770517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,2022-09-27,1585.774316,1132.547908,1581.774572,1585.774316,1585.774316,-0.141814,-0.141814,-0.141814,-0.000146,-0.000146,-0.000146,-0.141668,-0.141668,-0.141668,0.0,0.0,0.0,1360.889541
1784,2022-09-28,1580.027082,1130.532982,1620.616050,1580.027082,1580.027082,-0.135438,-0.135438,-0.135438,0.006675,0.006675,0.006675,-0.142112,-0.142112,-0.142112,0.0,0.0,0.0,1366.031693
1785,2022-09-29,1574.279848,1119.618668,1594.587957,1574.279848,1574.279848,-0.140789,-0.140789,-0.140789,0.000628,0.000628,0.000628,-0.141416,-0.141416,-0.141416,0.0,0.0,0.0,1352.639090
1786,2022-09-30,1568.532613,1097.958744,1556.120680,1568.532613,1568.532613,-0.144917,-0.144917,-0.144917,-0.005412,-0.005412,-0.005412,-0.139505,-0.139505,-0.139505,0.0,0.0,0.0,1341.225170


In [36]:
predict[["ds", "yhat_lower", "yhat_upper", "yhat"]].tail()

,ds,yhat_lower,yhat_upper,yhat
1783,2022-09-27,1132.547908,1581.774572,1360.889541
1784,2022-09-28,1130.532982,1620.616050,1366.031693
1785,2022-09-29,1119.618668,1594.587957,1352.639090
1786,2022-09-30,1097.958744,1556.120680,1341.225170
1787,2022-10-01,1112.457400,1563.271529,1346.572781


In [37]:
# Salvar modelo
with open('model_eth_usd.pckl', 'wb') as fout:
    pickle.dump(model, fout)